# Helpers

In [ ]:
%matplotlib widget
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from itertools import chain
import scipy
import seaborn as sns
sns.set_theme(style="darkgrid")
from vw_executor.vw import Vw
from vw_executor.loggers import ConsoleLogger


def new_ax():
    _,ax = plt.subplots(dpi=100, figsize=[9,4])
    return ax        

def _predictions_parse(lines):
    for l in lines:
        if ':' in l:
            yield {int(kv.split(':')[0]): float(kv.split(':')[1]) for kv in l.split(',')}

def predictions_2_df(files):
    return pd.DataFrame(_predictions_parse(chain(*map(lambda f: open(f), files))))

def plot(sim, result, i):
    job = result.iloc[i]['!Job']
    prediction_files = job.outputs['-p']
    df = pd.concat([sim, predictions_2_df(prediction_files)], axis=1)
    [df[(df['shared_good']==i)][j].rolling(window=100).mean().plot(
        figsize=[16,6], label=f'P(chosen={j} | Person={i})') for i in [0,1] for j in [0,1]]
    plt.legend(loc='best')

class Simulation:
    def __init__(self, n=10000, swap_after=5000, variance = 0, bad = 0):
        self.examples = []
        offset = 0
        for i in range(1, n):
            if i % swap_after == 0:
                offset = (offset + 1) % 2

            person = i % 2 # random.randint(0, 1)
            chosen = i % 4  // 2 #random.randint(0, 1)
            if (chosen + person + offset) % 2 == 0:
                reward =  0.7 + ((chosen + offset) % 2) * 0.1#int(random.random() < 0.7 + ((chosen + offset) % 2) * 0.1)
            else:
                reward = 0.3 - ((chosen + offset + 1) % 2) * 0.1 #int(random.random() < 0.3 - ((chosen + offset + 1) % 2) * 0.1)

            reward = reward + scipy.random.normal(0, variance)

            self.examples.append({
                "reward": reward,
                "shared_good": person,
                "a1_good": '0',
                "a2_good": '1',
                "chosen": chosen, 
                "prob": 0.5
            })
            for i in range(bad):
                self.examples[-1][f'a1_bad_{i}'] = random.random()
            self.examples[-1][f'a2_bad_{i}'] = random.random()
        self.df = pd.DataFrame(self.examples)
    
    def get(self):
        return pd.DataFrame(self.examples)
    
    def to_dsjson(self):
        for row in self.examples:
            o = {
                "_label_cost": -row['reward'],
                "_label_probability": row['prob'],
                "_label_Action": int(row['chosen'] + 1),
                "_labelIndex": int(row['chosen']),
                "a": [row['chosen'] + 1, (row['chosen'] + 1) % 2 + 1],
                "c": {'shared': {'f': str(row['shared_good'])},
                    '_multi': [{'a': {'f': str(row['a1_good'])}},
                        {'a': {'f': str(row['a2_good'])}}]},
                "p": [row['prob'], 1 - row['prob']] 
            }
            i = 0
            while True:
                if f'a1_bad_{i}' in row:
                    if 'b' not in o['c']['_multi'][0]: 
                        o['c']['_multi'][0]['b'] = {}
                        o['c']['_multi'][1]['b'] = {}   
                    o['c']['_multi'][0]['b'][f'a1_bad_{i}'] = row[f'a1_bad_{i}']
                    o['c']['_multi'][1]['b'][f'a2_bad_{i}'] = row[f'a1_bad_{i}']
                    i += 1
                else:
                    break
            yield json.dumps(o, separators=(",", ":"))

    def to_dsjson_file(self, fname):
        with open(fname, 'w') as f:
            for ex in self.to_dsjson():
                f.write(f'{ex}\n')             

    def visualize(self, window=100, ax = None):
        colors = ['green', 'red']
        styles = ['-', '-.']
        if ax is None:
            ax = (new_ax(), new_ax())
        [self.df[(self.df['shared_good']==i) & (self.df['chosen']==j)].rolling(window=window)['reward'].mean().plot(
            ax=ax[0], color = colors[j], style=styles[i], label=f'Person: {i}, chosen: {j}') for i in [0,1] for j in [0,1]]
        [self.df[self.df['chosen']==j].rolling(window=window)['reward'].mean().plot(
            ax=ax[1], color = colors[j], label=f'Chosen: {j}') for j in [0,1]]     
        ax[0].legend()
        ax[1].legend()
        
def _predictions_parse(lines):
    for l in lines:
        if ':' in l:
            yield {int(kv.split(':')[0]): float(kv.split(':')[1]) for kv in l.split(',')}

def predictions_2_df(files):
    return pd.DataFrame(_predictions_parse(chain(*map(lambda f: open(f), files))))

def plot(sim, result, i):
    colors = ['green', 'red']
    styles = ['-', '-.']
    job = result.iloc[i]['!Job']
    prediction_files = job.outputs['-p']
    df = pd.concat([sim, predictions_2_df(prediction_files)], axis=1)
    ax = new_ax()
    [df[(df['shared_good']==i)][j].rolling(window=100).mean().plot(
        ax=ax, color = colors[j], style=styles[i], label=f'P(chosen={j} | Person={i})') for i in [0,1] for j in [0,1]]
    ax.legend(loc='best')
    
def plot(sim, job, ax):
    colors = ['green', 'red']
    styles = ['-', '-.']
    prediction_files = job.outputs['-p']
    df = pd.concat([sim.get(), predictions_2_df(prediction_files)], axis=1)
    [(df[(df['shared_good']==i)][j].rolling(window=100).mean() + i * 0.05).plot(
        ax=ax, color = colors[j], style=styles[i], label=f'P(chosen={j} | Person={i})') for i in [0,1] for j in [0,1]]
    ax.legend(loc='best')
    
def generate_and_visualize(n=10000, swap_after=5000, variance=0, bad_log=0, window=100, ax=None, fig=None):
    sim = Simulation(n=n, swap_after=swap_after, variance = variance, bad = 2**bad_log)
    ax[0].clear()
    ax[1].clear()
    fig.suptitle('Data', fontsize=16)
    sim.visualize(window = window, ax=ax) 
    fig.canvas.draw_idle()
    
def generate_and_learn(n=10000, swap_after=5000, variance=0, bad_log=0, window=100, 
                       learning_rate=-20, coin=False, l1=-20, cb_type='ips', interactions='', ignore='', ignore_linear='', ax=None, fig=None):
    sim = Simulation(n=n, swap_after=swap_after, variance = variance, bad = 2**bad_log)
    ax[0][0].clear()
    ax[0][1].clear()
    ax[1][0].clear()
    ax[1][1].clear()

    fig.suptitle('Data', fontsize=16)
    sim.to_dsjson_file('small.json')

    opts = {'#base': '--cb_explore_adf --dsjson --epsilon 0.2',
            '#0': interactions,
            '#1': ignore,
            '#2': ignore_linear,
            '#lr': '--coin' if coin else f'--power_t 0 --learning_rate {2**learning_rate}',
            '--l1': 2**l1,
            '--cb_type': cb_type
           }

    result = Vw('vw', '.vw_cache', handlers=[],
                loggers=[
     #              ConsoleLogger('DEBUG')
                ]).train(['small.json'], opts, ['-p'])
    
    sim.visualize(window = window, ax=ax[0]) 
    plot(sim, result, ax[1][0])
    fig.canvas.draw_idle()
    

# Look at the data

In [ ]:
sim = Simulation(n=100, swap_after=50, variance = 0.1, bad = 2)
sim.get().head()

In [ ]:
Simulation(n=1000, swap_after=500, variance = 0.1, bad = 2).visualize(window=10)

# Interact

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12,7))

widget = interactive(
    generate_and_learn,
    n=IntSlider(min=1000, max=20000, step=1000, value=10000),
    swap_after=IntSlider(min=1000, max=20000, step=1000, value=5000),
    variance=FloatSlider(min=0, max=2, step=0.1, value=0),
    bad_log=IntSlider(min=0, max=10, step=1, value = 0),
    window=IntSlider(min=1, max=1000, step=10, value = 1),
    learning_rate=IntSlider(min=-20,max=10, step=1, value=-20),
    coin=Checkbox(),
    l1=IntSlider(min=-20,max=10, step=1, value=-20),
    cb_type=['ips', 'mtr'],
    interactions=['', '-q sa', '-q ab sb', '-q ::'],
    ignore=['', '--ignore b'],
    ignore_linear=['', '--ignore_linear sab'],
    ax=fixed(ax),
    fig=fixed(fig))

data_controls = HBox(widget.children[:4])
visualization_controls = HBox(widget.children[4:5])
learning_controls = HBox(widget.children[5:-4])
extra = HBox(widget.children[-4:-1])
output = widget.children[-1]
display(VBox([data_controls, visualization_controls, learning_controls, extra, output]))